<a href="https://colab.research.google.com/github/ShreyJ1729/EVA6-TSAI/blob/main/06-BatchNorm%26Regularization/06-BatchNorm%26Regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [53]:
class Net(nn.Module):
    def __init__(self, norm):
        super(Net, self).__init__()

        self.convblock1 = nn.Sequential(
            nn.Conv2d(1, 10, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(10) if norm=="BN" else nn.GroupNorm(2, 10) if norm=="GN" else nn.LayerNorm([10, 28, 28]),
            nn.Dropout2d(0.01),

            nn.Conv2d(10, 10, 3),
            nn.ReLU(),
            nn.BatchNorm2d(10) if norm=="BN" else nn.GroupNorm(2, 10) if norm=="GN" else nn.LayerNorm([10, 26, 26]),
            nn.Dropout2d(0.01),

            nn.Conv2d(10, 10, 3),
            nn.ReLU(),
            nn.BatchNorm2d(10) if norm=="BN" else nn.GroupNorm(2, 10) if norm=="GN" else nn.LayerNorm([10, 24, 24]),
            nn.Dropout2d(0.01),

            nn.MaxPool2d(2, 2)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(10) if norm=="BN" else nn.GroupNorm(2, 10) if norm=="GN" else nn.LayerNorm([10, 12, 12]),
            nn.Dropout2d(0.01),

            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(10) if norm=="BN" else nn.GroupNorm(2, 10) if norm=="GN" else nn.LayerNorm([10, 12, 12]),
            nn.Dropout2d(0.01),

            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(10) if norm=="BN" else nn.GroupNorm(2, 10) if norm=="GN" else nn.LayerNorm([10, 12, 12]),
            nn.Dropout2d(0.01),

            nn.MaxPool2d(2, 2)
        )
        
        self.convblock3 = nn.Sequential(
            nn.Conv2d(10, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16) if norm=="BN" else nn.GroupNorm(3, 16) if norm=="GN" else nn.LayerNorm([16, 6, 6]),
            nn.Dropout2d(0.01),

            nn.Conv2d(16, 15, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(15) if norm=="BN" else nn.GroupNorm(3, 15) if norm=="GN" else nn.LayerNorm([15, 6, 6]),
            nn.Dropout2d(0.01),
            
            nn.Conv2d(15, 10, 3, padding=1),
            nn.AvgPool2d(6)
        )

        self.fcblock = nn.Linear(10, 10)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = x.view(-1, 10)
        x = self.fcblock(x)
        return F.log_softmax(x, dim=1)

def model(norm):
  return Net(norm)

In [54]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
net = model("BN").to(device)
summary(net, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
              ReLU-2           [-1, 10, 28, 28]               0
       BatchNorm2d-3           [-1, 10, 28, 28]              20
         Dropout2d-4           [-1, 10, 28, 28]               0
            Conv2d-5           [-1, 10, 26, 26]             910
              ReLU-6           [-1, 10, 26, 26]               0
       BatchNorm2d-7           [-1, 10, 26, 26]              20
         Dropout2d-8           [-1, 10, 26, 26]               0
            Conv2d-9           [-1, 10, 24, 24]             910
             ReLU-10           [-1, 10, 24, 24]               0
      BatchNorm2d-11           [-1, 10, 24, 24]              20
        Dropout2d-12           [-1, 10, 24, 24]               0
        MaxPool2d-13           [-1, 10, 12, 12]               0
           Conv2d-14           [-1, 10,

In [55]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [67]:
import torch

def fit(model, device, train_loader, val_loader, optimizer, epochs, loss_func, lambda_l1=0, l1reg=None, scheduler=None, notebook=False, leave=False):
    model = model.to(device)

    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": [],
    }

    if notebook:
        from tqdm.notebook import tqdm
    else:
        from tqdm import tqdm

    for epoch in range(epochs):
        model.train()
        print(f"Epoch {epoch}")
        
        epoch_results = {
            "train_loss": [],
            "train_acc": [],
        }
        epoch_losses = []
        correct = 0

        for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)

            l1_loss = 0
            if l1reg is not None:
              l1 = 0
              for p in model.parameters():
                l1 = l1 + p.abs().sum()
              l1_loss = lambda_l1 * l1
              loss = loss + l1_loss
              
            loss.backward()
            optimizer.step()
            
            epoch_losses.append(loss.item())
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

            # tqdm.write(f'\r[Epoch {epoch}] loss={round(sum(epoch_losses)/len(epoch_losses), 4)}', end="")
        tqdm.write('\r[Epoch {}]\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(epoch, 
            round(sum(epoch_losses)/len(epoch_losses), 4), correct, len(train_loader.dataset),
            100. * correct / len(train_loader.dataset)))
        scheduler.step()
        
        results["train_loss"].append(round(sum(epoch_losses)/len(epoch_losses), 4))
        results["train_acc"].append(100. * correct / len(train_loader.dataset))
        
        if scheduler is not None:
          scheduler.step()

        # validation
        model.eval()
        test_loss = 0
        correct = 0

        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
                pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)

        tqdm.write("Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset) )
        )

        results["test_loss"].append(test_loss)
        results["test_acc"].append(100. * correct / len(test_loader.dataset) )

    return results

In [ ]:
#Training all models

model = Net(norm).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.85)
l1reg = False

final = []

for norm in ["BN", "GN", "LN"]:
  print(f"Training {norm} network...")
  l1reg = False
  if norm=="BN":
    l1reg = True
  results = fit(model, device, train_loader, test_loader, optimizer, epochs=20, loss_func=F.nll_loss, lambda_l1 = 0.01, l1reg=True if norm=="BN" else False,
      scheduler=scheduler, notebook=True, leave=True)
  final.append(results)

Training BN network...
Epoch 0



[Epoch 0]
Train set: Average loss: 3.0162, Accuracy: 47140/60000 (78.57%)
Test set: Average loss: 0.3021, Accuracy: 9162/10000 (91.62%)

Epoch 1



[Epoch 1]
Train set: Average loss: 1.8055, Accuracy: 55005/60000 (91.67%)
Test set: Average loss: 0.5213, Accuracy: 8482/10000 (84.82%)

Epoch 2



[Epoch 2]
Train set: Average loss: 1.3667, Accuracy: 55229/60000 (92.05%)
Test set: Average loss: 1.3096, Accuracy: 5260/10000 (52.60%)

Epoch 3



[Epoch 3]
Train set: Average loss: 1.2053, Accuracy: 55500/60000 (92.50%)
Test set: Average loss: 0.8050, Accuracy: 6889/10000 (68.89%)

Epoch 4


...


[Epoch 19]
Train set: Average loss: 0.6873, Accuracy: 58251/60000 (97.08%)
Test set: Average loss: 0.0696, Accuracy: 9854/10000 (98.54%)

{'train_loss': [3.0162, 1.8055, 1.3667, 1.2053, 1.1101, 1.0312, 0.9677, 0.9183, 0.8819, 0.8501, 0.8125, 0.7881, 0.764, 0.7461, 0.7314, 0.7184, 0.7108, 0.6992, 0.6924, 0.6873], 'train_acc': [78.56666666666666, 91.675, 92.04833333333333, 92.5, 93.25833333333334, 93.85666666666667, 94.27833333333334, 94.80166666666666, 94.91166666666666, 95.26833333333333, 95.69, 95.845, 96.13833333333334, 96.40333333333334, 96.55, 96.65333333333334, 96.73, 96.895, 97.02666666666667, 97.085], 'test_loss': [0.302061159992218, 0.521256687784195, 1.3095779121398925, 0.8049688955307007, 0.458683171415329, 0.3907677990436554, 0.44134433821439745, 0.16343357033729553, 0.37155115256309507, 0.1505669280409813, 0.3103889931678772, 0.2850452582359314, 0.23830972331762315, 0.29119927093982695, 0.10466178994178772, 0.13521597385406495, 0.08040216374993324, 0.1292358836531639, 0.06


[Epoch 0]
Train set: Average loss: 0.6830, Accuracy: 58265/60000 (97.11%)
Test set: Average loss: 0.0814, Accuracy: 9834/10000 (98.34%)

Epoch 1



[Epoch 1]
Train set: Average loss: 0.6806, Accuracy: 58322/60000 (97.20%)
Test set: Average loss: 0.0662, Accuracy: 9856/10000 (98.56%)

Epoch 2



[Epoch 2]
Train set: Average loss: 0.6781, Accuracy: 58316/60000 (97.19%)
Test set: Average loss: 0.0674, Accuracy: 9863/10000 (98.63%)

Epoch 3



[Epoch 3]
Train set: Average loss: 0.6739, Accuracy: 58398/60000 (97.33%)
Test set: Average loss: 0.0620, Accuracy: 9879/10000 (98.79%)

Epoch 4


In [ ]:
# Test loss plot:
# 0 - BN
# 1 - GN
# 2 - LN

import matplotlib.pyplot as plt
plt.plot(final[0]["test_loss"], label="BN+L1")
plt.plot(final[1]["test_loss"], label="GN")
plt.plot(final[2]["test_loss"], label="LN")
plt.legend()
plt.xlabel("epochs")
plt.ylabel("loss (F.nll_loss)")
plt.title("Validation Loss vs Epochs")
plt.show()

plt.plot(final[0]["test_acc"], label="BN+L1")
plt.plot(final[1]["test_acc"], label="GN")
plt.plot(final[2]["test_acc"], label="LN")
plt.legend()
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.title("Accuracy vs Epochs")
plt.show()
